In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from kan import *
from kan.utils import ex_round

In [3]:
data0=pd.read_csv('/home/sam/Documents/pima-indians-diabetes.csv')
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Number of times pregnant                       768 non-null    int64  
 1   Plasma glucose concentration                   768 non-null    int64  
 2   Diastolic blood pressure (mmHg)                768 non-null    int64  
 3   Triceps skin fold thickness (mm)               768 non-null    int64  
 4   2-hour serum insulin (μIU/ml)                  768 non-null    int64  
 5   Body mass index (weight in kg/(height in m)2)  768 non-null    float64
 6   Diabetes pedigree function                     768 non-null    float64
 7   Age                                            768 non-null    int64  
 8   Diabetes onset                                 768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
data0.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mmHg),Triceps skin fold thickness (mm),2-hour serum insulin (μIU/ml),Body mass index (weight in kg/(height in m)2),Diabetes pedigree function,Age,Diabetes onset
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
data=data0.to_numpy()
x=torch.tensor(data[:,0:8],dtype=torch.float32)
y=torch.tensor(data[:,8],dtype=torch.float32).reshape(-1,1)

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size =.12, random_state = 42)
data={'train_input':xtrain,'test_input':xtest,'train_label':ytrain,'test_label':ytest}

In [7]:
#defining the model 
model = KAN(width=[8,20,1], grid=5, k=2, seed=42)

checkpoint directory created: ./model
saving model version 0.0


In [8]:
model.fit(data, opt="LBFGS", steps=50, lamb=0.01)

| train_loss: 3.83e-01 | test_loss: 4.16e-01 | reg: 5.07e+00 | : 100%|█| 50/50 [00:10<00:00,  4.59it

saving model version 0.1


{'train_loss': [array(0.48491386, dtype=float32),
  array(0.42202488, dtype=float32),
  array(0.40295613, dtype=float32),
  array(0.39725056, dtype=float32),
  array(0.40068397, dtype=float32),
  array(0.40883434, dtype=float32),
  array(0.3958487, dtype=float32),
  array(0.39710552, dtype=float32),
  array(0.39778143, dtype=float32),
  array(0.39967537, dtype=float32),
  array(0.3927776, dtype=float32),
  array(0.39189166, dtype=float32),
  array(0.38759464, dtype=float32),
  array(0.38824344, dtype=float32),
  array(0.3847913, dtype=float32),
  array(0.3846643, dtype=float32),
  array(0.3850672, dtype=float32),
  array(0.38527754, dtype=float32),
  array(0.38536918, dtype=float32),
  array(0.3844263, dtype=float32),
  array(0.3851151, dtype=float32),
  array(0.38520768, dtype=float32),
  array(0.38499105, dtype=float32),
  array(0.3849805, dtype=float32),
  array(0.38380975, dtype=float32),
  array(0.38387138, dtype=float32),
  array(0.3835571, dtype=float32),
  array(0.3833661, dtyp

In [10]:
model(xtest[0:15])

tensor([[ 0.3720],
        [ 0.2954],
        [ 0.1985],
        [ 0.1854],
        [ 0.5005],
        [ 0.4622],
        [-0.0606],
        [ 0.6251],
        [ 0.6418],
        [ 0.6673],
        [ 0.3068],
        [ 0.8202],
        [ 0.4258],
        [ 0.1544],
        [-0.0323]], grad_fn=<AddBackward0>)

In [11]:
#unlike a sigmoid KAN does not flatten easily at 1 or 0, thus a threshold function is needed
def thres(data,thres=.7):
    n=len(data)
    for i in range(n):
        if data[i][0]>=thres:
            data[i][0]=1.0
        else:
            data[i][0]=0.0
    return(data)

yn=thres(model(xtest))

accuracy = (yn.round() == ytest).float().mean()
print(f'Accuracy: {accuracy}')

Accuracy: 0.7634408473968506
